# EfficientNet Paper

The EfficientNet paper [1] proposed a new model scalling method which scales the depth, width and resolution at the same time. These parameters are usually tuned independently, Ref. [1] claims tuning them together is more efficient.

First, we study the implementation from the [official repo](https://github.com/tensorflow/tpu/tree/master/models/official/efficientnet).

The model is defined by a couple of parameters: $\alpha$ (depth coeff), $\beta$ (width coeff), $\gamma$ (resolution coeff), and $\phi$ (compound coeff). To tune it, the authors first do a small grid search of $\alpha$, $\beta$, $\gamma$, with $\phi$ fixed; then fix $\alpha$, $\beta$, $\gamma$ and scale up with different $\phi$. 

The architecture of the baseline model (EfficientNet-B0) is shown in Table 1 of Ref. [1]:

![EfficientNet paper Table 1](en_b0.png)

The parameters for the `MBConv` blocks are defined in `efficientnet`:

In [ ]:
blocks_args = [
  'r1_k3_s11_e1_i32_o16_se0.25', 'r2_k3_s22_e6_i16_o24_se0.25',
  'r2_k5_s22_e6_i24_o40_se0.25', 'r3_k3_s22_e6_i40_o80_se0.25',
  'r3_k5_s11_e6_i80_o112_se0.25', 'r4_k5_s22_e6_i112_o192_se0.25',
  'r1_k3_s11_e6_i192_o320_se0.25',
]

In here, we take an example, `'r2_k3_s22_e6_i16_o24_se0.25'` means:

|r2|k3|s22|e6|i16|o24|se0.25|
|-|-|-|-|-|-|-|
|repeat=2|kernel size=3|stride=(2,2)|expand_ratio=6|input_filters=16|output_filters=24|SE_ratio=0.25|


So `blocks_args` defines all the necessary parameters for stage 2-8, the `MBConv` blocks, in Table 1. 

The scale parameters (determined by $\alpha$, $\beta$, $\gamma$, $\phi$) for all models is defined in `efficientnet_params`:

In [ ]:
params_dict = {
  # (width_coefficient, depth_coefficient, resolution, dropout_rate)
  'efficientnet-b0': (1.0, 1.0, 224, 0.2),
  'efficientnet-b1': (1.0, 1.1, 240, 0.2),
  'efficientnet-b2': (1.1, 1.2, 260, 0.3),
  'efficientnet-b3': (1.2, 1.4, 300, 0.3),
  'efficientnet-b4': (1.4, 1.8, 380, 0.4),
  'efficientnet-b5': (1.6, 2.2, 456, 0.4),
  'efficientnet-b6': (1.8, 2.6, 528, 0.5),
  'efficientnet-b7': (2.0, 3.1, 600, 0.5),
}

With `blocks_args` and `params_dict`, we have enough information to build the model. 

Now what exactly is the `MBConv` block? 

## Mobile Inverted Residual Blocks

The `MBConv` block used in here is introduced in Ref. [2]. The core part is:

![Mobile inverted residual paper Fig. 3](mb_inv.png)

In terms of Ref. [1], the relevant code is in the `MBConvBlock` `_call_se`:

In [ ]:
x = self._relu_fn(self._bn0(self._expand_conv(inputs), training=training))
x = self._relu_fn(self._bn1(self._depthwise_conv(x), training=training))
x = self._bn2(self._project_conv(x), training=training)
x = tf.add(x, inputs)

The first step is to expand: increase the number of channels while keep the width and height unchanged. The second step is to do a depth-wise convolution, which changes the width and height, and the number of channels stays the same (depth-wise: convolution for each channel is calculated independently). The third step is to reduce the number of channels while keeping the width and height unchanged. Finally, the skip connection is added as usual.

It's called "inverted", because the input and output layer are thin, while the layers in between are thick, which is the opposite of usual network structures. 

## Squeeze-and-Excitation Networks

The idea of SENet is to add weights to the channels. How do we know the weights? Similar to attention, we add parameters and train them. This technique can be apply to enhance existing architectures. The following figure shows how ResNet can be turned into SE-ResNet:

![SE-Net paper Fig. 3](senet.png)

In Ref. [1], SE is added on top of the `MBConv` block. The relevant code is in the `MBConvBlock` `_call_se`:

In [ ]:
se_tensor = tf.reduce_mean(input_tensor, self._spatial_dims, keepdims=True)
se_tensor = self._se_expand(self._relu_fn(self._se_reduce(se_tensor)))
tf.sigmoid(se_tensor) * input_tensor

The first step is to do an average pooling, which reduces the width and height into 1. The number of channels is unchanged. The second step is to squeeze: reduce the number of channels.  Then it passes through an activation function. The next step step is to expand: increase the number of channels back to that of the input. At this point, we got `c` numbers (`c` is the number of channels), which is then used to calculate the weights of the input channels.

## Reference

[1] [EfficientNet: Rethinking Model Scaling for Convolutional Neural Networks](https://arxiv.org/abs/1905.11946v2)

[2] [MobileNetV2: Inverted Residuals and Linear Bottlenecks](https://arxiv.org/abs/1801.04381v4)

[3] [Squeeze-and-Excitation Networks](https://arxiv.org/abs/1709.01507v4)